# WellSAP

## Personal WellBeing Training

### Imports

In [17]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

### Data Loading

In [2]:
hr_data = pd.read_csv('./../dataset/hr_data.csv',header=0, index_col='Employee_ID')
attrition_data = pd.read_csv('./../dataset/attrition_data.csv', header=0, index_col='Employee_ID')
lifestyle_data = pd.read_csv('./../dataset/lifestyle_data.csv', header=0, index_col='Employee_ID')

### Data Pre-Processing

#### Lifestyle Data Pre-Processing

In [3]:
input_features_classes = lifestyle_data.loc[:,['FRUITS_VEGGIES', 'DAILY_STRESS', 'BMI_RANGE', 'DONATION']]
input_features_int_int = lifestyle_data.loc[:,['PLACES_VISITED', 'CORE_CIRCLE', 'SUPPORTING_OTHERS', 'SOCIAL_NETWORK']]
input_features_int_float = lifestyle_data.loc[:,['SLEEP_HOURS', 'WEEKLY_MEDITATION', 'DAILY_SHOUTING']]

output_features = lifestyle_data.loc[:,'WORK_LIFE_BALANCE_SCORE']

In [30]:
input_features_classes = input_features_classes.groupby('Employee_ID').max().astype(int)
input_features_int_int = input_features_int_int.groupby('Employee_ID').mean().apply(np.ceil).astype(int)
input_features_int_float = input_features_int_float.groupby('Employee_ID').mean()

In [31]:
input_features = pd.concat([input_features_classes, input_features_int_int, input_features_int_float], axis=1)
input_features

,FRUITS_VEGGIES,DAILY_STRESS,BMI_RANGE,DONATION,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,SLEEP_HOURS,WEEKLY_MEDITATION,DAILY_SHOUTING
Employee_ID,,,,,,,,,,,
1.0,5,5,2,5,6,7,5,7,7.00,7.28,2.08
2.0,5,5,2,5,5,5,5,7,6.76,6.52,3.88
3.0,5,5,2,5,6,6,6,8,6.92,5.52,3.16
4.0,5,5,2,5,7,6,6,7,7.52,6.20,2.00
5.0,5,5,2,5,5,6,5,6,7.64,5.92,2.92
...,...,...,...,...,...,...,...,...,...,...,...
307.0,5,4,2,5,6,6,5,6,7.12,6.00,1.76
308.0,5,5,2,5,5,6,7,7,6.92,5.80,3.08
309.0,5,5,2,5,5,7,8,7,6.80,4.68,3.08


#### HR and Attrition Data Pre-Processing

In [32]:
gender_encoder = preprocessing.LabelEncoder()
attrition_data['Gender'] = gender_encoder.fit_transform(attrition_data['Gender'])
age_encoder = preprocessing.LabelEncoder()
hr_data['Age_Group'] = age_encoder.fit_transform(hr_data['Age_Group'])

In [33]:
input_features = pd.concat([input_features, hr_data['MaritalStatusID'], hr_data['Age_Group'], attrition_data["Gender"], attrition_data['RelationshipSatisfaction'].astype(int)], axis=1)
input_features

,FRUITS_VEGGIES,DAILY_STRESS,BMI_RANGE,DONATION,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,SLEEP_HOURS,WEEKLY_MEDITATION,DAILY_SHOUTING,MaritalStatusID,Age_Group,Gender,RelationshipSatisfaction
Employee_ID,,,,,,,,,,,,,,,
1.0,5,5,2,5,6,7,5,7,7.00,7.28,2.08,0,0,1,2
2.0,5,5,2,5,5,5,5,7,6.76,6.52,3.88,1,1,1,1
3.0,5,5,2,5,6,6,6,8,6.92,5.52,3.16,1,0,0,1
4.0,5,5,2,5,7,6,6,7,7.52,6.20,2.00,1,0,0,3
5.0,5,5,2,5,5,6,5,6,7.64,5.92,2.92,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307.0,5,4,2,5,6,6,5,6,7.12,6.00,1.76,0,0,1,2
308.0,5,5,2,5,5,6,7,7,6.92,5.80,3.08,0,0,0,3
309.0,5,5,2,5,5,7,8,7,6.80,4.68,3.08,0,1,0,2


#### Output Data Pre-Processing

In [22]:
output_scaler = preprocessing.StandardScaler()
output_features.reset_index(inplace=True)
output_features['WORK_LIFE_BALANCE_SCORE'] = output_scaler.fit_transform(output_features['WORK_LIFE_BALANCE_SCORE'])
output_features

TypeError: Cannot reset_index inplace on a Series to create a DataFrame